In [ ]:
using ValueShapes
using ArraysOfArrays
using StatsBase 
using LinearAlgebra
using Statistics
using BAT
using Distributions 
using IntervalSets

using HCubature

using Plots
using Colors
using ColorSchemes
using LaTeXStrings

pyplot(size=(750,500))
line_colors = ColorSchemes.tab20b;

In [ ]:
import PyPlot
plt = PyPlot

In [ ]:
density_type = Normal(0.0, 1.0)
pdf_density(x; density_type=density_type) = pdf(density_type,x...)

prior = NamedTupleDist(x =[-10.0 .. 10.0] )
Norm_prior = 20

log_likelihood = let f = pdf_density
    params ->  LogDVal(log(pdf_density(params.x)))
end

posterior = PosteriorDensity(log_likelihood, prior)

nsamples = 10^3
nchains = 3

function gen_mcmc_samples(n_samples::Int64, n_chains::Int64; posterior=posterior, norm_prior=Norm_prior)
    samples_tmp = bat_sample(posterior, (n_samples, n_chains), MetropolisHastings()).result
    samples_vector_tmp = flatview(unshaped.(samples_tmp.v))[1,:]
    samples_likelihoods = exp.(samples_tmp.logd) * norm_prior
    samples_weights = samples_tmp.weight
    return (samples_vector_tmp, samples_likelihoods, samples_weights)
end

function gen_iid_samples(n_samples::Int64, n_chains::Int64; density_type=density_type)
    samples_vector_tmp = rand(density_type, n_samples*n_chains)
    samples_likelihoods = pdf.(density_type, samples_vector_tmp)
    samples_weights = ones(length(samples_vector_tmp))
    return (samples_vector_tmp, samples_likelihoods, samples_weights)
end

function generate_experiments(x_r::StepRangeLen, gen_samples::Function; n_samples=nsamples, n_chains=nchains, n_trials=15000) # for the paper use 15000
    
    r_estimates = Vector{Float64}()
    r_bias = Vector{Float64}()
    hm_estimates = Vector{Float64}()
    hm_bias = Vector{Float64}()
    
    for i_trial in 1:n_trials # generate new MCMC samples 
        (samples_vector_tmp, samples_likelihoods, samples_weights) = gen_samples(n_samples, n_chains)
        for x in x_r #change window size

            mask = -x .< samples_vector_tmp .< x
            V_run = 2*x 
            
            r_hat = sum(samples_weights[mask]) /sum(samples_weights)
            
            r_unweighted = sum(mask)/length(samples_weights)
            r_b = (1-r_unweighted)/sum(mask)
    
            push!(r_estimates, r_hat)
            push!(r_bias, r_b)
            
            μ_array = 1 ./ samples_likelihoods[mask]
            μ_var =  var(μ_array, weights(samples_weights[mask]))/sum(samples_weights[mask])
            μ_mean = mean(μ_array, weights(samples_weights[mask]))
            
            push!(hm_estimates, V_run/μ_mean)
            push!(hm_bias, μ_var/(μ_mean^2))

        end
        
    end
    
    hm_estimates = reshape(hm_estimates, length(x_range), n_trials)
    r_estimates = reshape(r_estimates, length(x_range), n_trials)
    r_bias = reshape(r_bias, length(x_range), n_trials)
    hm_bias = reshape(hm_bias, length(x_range), n_trials)
    
    return hm_estimates, r_estimates, r_bias, hm_bias
end

# Normal Distribution: 

In [ ]:
x_range = range(0.01, stop=4.5, length=60)

# Generate Experiments: 

# MCMC
# hm_estimates, r_estimates, bias_1, bias_2  = generate_experiments(x_range, gen_mcmc_samples); #3-4 minutes

# iid
hm_estimates, r_estimates, bias_1, bias_2  = generate_experiments(x_range, gen_iid_samples); #3-4 minutes

int_tmp = hm_estimates./r_estimates
mean_int_mcmc = mean(int_tmp, dims=2)[:,1]
std_int_mcmc = std(int_tmp, dims=2)[:,1];

n_run_mean = mean(3000 .* r_estimates, dims=2)[:,1]

int_tmp_bias = hm_estimates./r_estimates.*(1 .- bias_1 .- bias_2)
mean_bc_mcmc = mean(int_tmp_bias, dims=2)[:,1]
std_bc_mcmc = std(int_tmp_bias, dims=2)[:,1];

In [ ]:
# Cutt-off positions: 

Ind_1 = 4
Ind_2 = 22
Ind_3 = 56

lc1 = line_colors[6]
lc2 = line_colors[14]
lc3 = line_colors[10]

bins_integrals = range(minimum(int_tmp), stop = maximum(int_tmp), length = 50)

histogram_1 = fit(Histogram, int_tmp[Ind_1,:], nbins=30);
histogram_1 = normalize(histogram_1, mode=:density);

histogram_2 = fit(Histogram, int_tmp[Ind_2,:], nbins=30);
histogram_2 = normalize(histogram_2, mode=:density);

histogram_3 = fit(Histogram, int_tmp[Ind_3,:], nbins=30);
histogram_3 = normalize(histogram_3, mode=:density);

mean_int = mean_int_mcmc
std_int = std_int_mcmc;

In [ ]:
SMALL_SIZE = 12
MEDIUM_SIZE = 13
BIGGER_SIZE = 13

plt.rc("font", size=SMALL_SIZE)          # controls default text sizes
plt.rc("axes", titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc("axes", labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc("xtick", labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc("ytick", labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)    # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
x_points= x_range
y_points = 2. .* pdf_density.(x_points)

fig, ax = plt.subplots(1,2, figsize=(13, 6))
fig.subplots_adjust(wspace=0.2)

ax_right =  ax[1] 
ax_left = ax[1].twinx()
ax2_left = ax[2]

#ax_left.hist(abs.(samples_vector), weights = samples_weights, bins=100, alpha=0.1, color="k", density=true)
ax_left.fill_between(x_points, zeros(length(x_points)), y_points, hatch="///", facecolor="lightgray",edgecolor="white", alpha=1, linewidth=0.0, label="Density")
ax_left.legend(facecolor="white", framealpha=1, bbox_to_anchor=(0.93,1.0))

ax_right.fill_between(x_range, mean_int .- std_int, mean_int .+ std_int, alpha=1, color=plt.cm.Blues(0.3),  label=L"Std.: $\sigma(\hat{I})$")
ax_right.plot(x_range, ones(length(x_range)), c=:red, label="Truth")
ax_right.plot(x_range, mean_int, label=L"Mean: $\langle\hat{I}\rangle$", color=plt.cm.Blues(0.9))
ax[1].set_xlabel(L"$|x|$")
ax_right.set_ylabel("Ratio to Truth")
ax_left.set_ylabel(L"$p(|x|)$", color="gray")
ax_left.tick_params(axis="y", labelcolor="gray", )
ax_left.set_ylim(0,1.1)
ax[1].set_xlim(0, x_range[end])


ax_right.axvline(x_range[Ind_1], c="k", ls=":", lw=1)
ax_right.axvline(x_range[Ind_2], c="k", ls="-", lw=1)
ax_right.axvline(x_range[Ind_3], c="k", ls="--", lw=1)

ax_right.legend(loc="upper left", facecolor="white", framealpha=1)
#ax_left.legend(loc="upper left", frameon=true, framealpha=1)

ax2_left.axvline(1, c=:red)
ax2_left.hist(int_tmp[Ind_1,:], histtype="step", bins=30, density=true, ls=":",color="k", label="Window 1")
ax2_left.hist(int_tmp[Ind_2,:], histtype="step", bins=30, density=true, ls="-",color="k", label="Window 2")
ax2_left.hist(int_tmp[Ind_3,:], histtype="step", bins=30, density=true, ls="--",color="k", label="Window 3")

ax2_left.legend(loc="upper left", frameon=true, framealpha=0.8, ncol=1)

ax2_left.get_yaxis().set_visible(false)
ax2_left.set_xlabel(L"Integral Estimate $\hat{I}$")


ax_right.set_zorder(1)  # default zorder is 0 for ax1 and ax2
ax_right.patch.set_visible(false) 

# fig.savefig("../../AHMI_publication/NormalDistributionData/ahmi_1D.pdf", bbox_inches = "tight")

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(13, 5))

fig.subplots_adjust(wspace=0.4)

ax1 = ax[1]
ax2 = ax1.twinx()
ax3 = ax[2]

# ax1.fill_between(x_range, mean_bc_mcmc .- std_bc_mcmc, mean_int .+ std_bc_mcmc, alpha=0.4, color="lightblue",  label=L"Std.: $\sigma(\hat{I})$")
# ax1.fill_between(x_range, mean_int_mcmc .- std_int_mcmc, mean_int_mcmc .+ std_int_mcmc, alpha=0.4, color="darkblue",  label=L"Std.: $\sigma(\hat{I})$")
ax1.plot(x_range, ones(length(x_range)), c=:red, label="Truth")
ax1.plot(x_range, mean_bc_mcmc, color=plt.cm.Blues(0.9), label=L"Mean: $\langle\hat{I}\rangle$ w/ Correction")
ax1.plot(x_range, mean_int_mcmc,color=plt.cm.Blues(0.9), ls="--", label=L"Mean: $\langle\hat{I}\rangle$ w/o Correction" )

#ax_left.hist(abs.(samples_vector), weights = samples_weights, bins=100, alpha=0.1, color="k", density=true)
ax2.fill_between(x_points, zeros(length(x_points)), y_points, hatch="///", facecolor="lightgray",edgecolor="white", alpha=1, linewidth=0.0, label="Density")
ax2.legend(loc=0)

# ax2.fill_between(x_points, zeros(length(x_points)), y_points, alpha=0.2, color="gray", label="Density Function")
ax2.set_ylabel(L"$p(|x|)$", color="gray")
ax2.tick_params(axis="y", labelcolor="gray", )
ax2.set_ylim(0,1.0)
ax2.set_xlim(0,4.5)

ax1.set_ylabel("Ratio to Truth")
ax1.set_xlabel(L"$|x|$")

ax1.legend(loc="upper left", frameon=true, framealpha=1, facecolor="white")

ax3.plot(x_points, 1 .- mean(bias_1, dims=2)[:,1], ls="-.", color=plt.cm.Blues(0.9), label=L"Mean: Bias From $\hat{r}$")
ax3.plot(x_points, 1 .- mean(bias_2, dims=2)[:,1], ls=":", color=plt.cm.Blues(0.9), label=L"Mean: Bias From $\hat{x}$")
ax3.set_xlabel(L"$|x|$")
ax3.set_ylabel("1 - Bias")
ax3.legend(loc="upper center", frameon=true, framealpha=1, facecolor="white")
ax3.set_xlim(0,4.5)
ax3.set_ylim(0.96,1.02)

# ax3.plot(x_points, n_run_mean, color=plt.cm.Blues(0.9),)
# ax3.set_xlabel(L"$|x|$")
# ax3.set_ylabel(L"$<N>$ Samples Inside Window")
# ax3.set_xlim(0,4.5)
# ax3.set_ylim(0,3200)

ax1.set_zorder(1)  # default zorder is 0 for ax1 and ax2
ax1.patch.set_visible(false) 

plt.show()

# fig.savefig("../../AHMI_publication/NormalDistributionData/bias_correction.pdf", bbox_inches = "tight")